## Overview

Animated characters, objects, or backgrounds have been an essential part of videogames. Many iconic games, such as Super Mario Bros, have used sprite animation to bring their characters to life, and make their products more engaging and visually appealing. This blog is a brief exploration of how javascript is used to create your own simple animations using sprites. 

### Spritesheet Deinfition

In order to create the animation, you need to find an image which contains each individual frame side by side, also known as a spritesheet.

> Animation is achieved by quickly sequencing through a series of still images in code, allowing the images to appear as if they are moving.

### Spritesheet Example

This spritesheet is used for a player in the game.  Each row represents a different orientation of movement left-right-up-down.

![chillguy]({{site.baseurl}}/images/gamify/chillguy.png)

> It is an option to make your own Spritesheet, many students have used [Pixel Art](https://www.pixilart.com/)

### Game Definitions for Character 
The code snippet below outlines the data for sprite character definition, including the location of the sprite, handling different animation for left-right-up-down, defining action keys, position, direction, and hitbox.


```js
    // Player data for Chillguy
    const sprite_src_chillguy = path + "/images/gamify/chillguy.png"; // The Sprite
    const CHILLGUY_SCALE_FACTOR = 5; // Sizing character, guess and check by altering value
    const sprite_data_chillguy = { // Encapsulates an Object Literal for the Character's properties
        id: 'Chill Guy',
        greeting: "Hi I am Chill Guy, the desert wanderer. I am looking for wisdom and adventure!",
        src: sprite_src_chillguy,
        SCALE_FACTOR: CHILLGUY_SCALE_FACTOR,
        STEP_FACTOR: 1000, // How many steps across the screen
        ANIMATION_RATE: 50, // Speed up or slow down animation
        INIT_POSITION: { x: 0, y: height - (height/CHILLGUY_SCALE_FACTOR) }, // Position is relative
        pixels: {height: 384, width: 512},
        orientation: {rows: 3, columns: 4 },
        down: {row: 0, start: 0, columns: 3 },
        downRight: {row: 1, start: 0, columns: 3, rotate: Math.PI/16 },
        downLeft: {row: 2, start: 0, columns: 3, rotate: -Math.PI/16 },
        left: {row: 2, start: 0, columns: 3 },
        right: {row: 1, start: 0, columns: 3 },
        up: {row: 3, start: 0, columns: 3 },
        upLeft: {row: 2, start: 0, columns: 3, rotate: Math.PI/16 },
        upRight: {row: 1, start: 0, columns: 3, rotate: -Math.PI/16 },
        hitbox: { widthPercentage: 0.45, heightPercentage: 0.2 },
        keypress: { up: 87, left: 65, down: 83, right: 68 } // W, A, S, D
    };
```

This code snippet below shows how to association **data: sprite_data_chillguy** object literal with the **class: Player** object-orieted programing template.  Each of these class to data associations become canvas elements and objects in the game.
```js
    // List of objects defnitions for this level
    this.classes = [
      { class: Background, data: image_data_desert },
      { class: Player, data: sprite_data_chillguy },
      { class: Npc, data: sprite_data_tux },
      { class: Npc, data: sprite_data_octocat },
      { class: Npc, data: sprite_data_robot },
      { class: Npc, data: sprite_data_r2d2 },
      { class: Npc, data: sprite_data_stocks },
      { class: Npc, data: sprite_data_crypto}
    ];
```

### Player extends Character
The constructor for Character which is the parent class for Player using the data from the object literal to define a Player object.

- Game Canvas: Assigns properties to canvas element within the constructor of the Character class.
```js
    // Create canvas element
    this.canvas = document.createElement("canvas");
    this.canvas.id = data.id || "default";
    this.canvas.width = data.pixels?.width || 0;
    this.canvas.height = data.pixels?.height || 0;
    this.hitbox = data?.hitbox || {};
    this.ctx = this.canvas.getContext('2d');
    document.getElementById("gameContainer").appendChild(this.canvas);
```

-  Player Object: Assigns properties to the Player object within the extended Character constructor.
```js
    // Initialize the object's scale based on the game environment
    this.scale = { width: this.gameEnv.innerWidth, height: this.gameEnv.innerHeight };
    
    // Check if sprite data is provided
    if (data && data.src) {
        this.scaleFactor = data.SCALE_FACTOR || SCALE_FACTOR;
        this.stepFactor = data.STEP_FACTOR || STEP_FACTOR;
        this.animationRate = data.ANIMATION_RATE || ANIMATION_RATE;
        this.position = data.INIT_POSITION || INIT_POSITION;

        // Load the sprite sheet
        this.spriteSheet = new Image();
        this.spriteSheet.src = data.src;

        // Initialize animation properties
        this.frameIndex = 0; // index reference to current frame
        this.frameCounter = 0; // count each frame rate refresh
        this.direction = 'down'; // Initial direction
        this.spriteData = data;
    } else {
        throw new Error('Sprite data is required');
    }
```


### Game Draw for Sprint
The draw() method is a crucial part of the animation process in our game. It is responsible for rendering the game character on the canvas, either using a sprite sheet for more complex animations or a simple red square as a placeholder. This method ensures that the character is drawn correctly based on its current state and position. Below is the implementation of the draw() method in Character.js.

![InspectChillGuy]({{site.baseurl}}/images/gamify/chillguy_inspect.png)


```js
/**
 * Draws the object on the canvas.
 * 
 * This method renders the object using the sprite sheet if provided, otherwise a red square.
 */
draw() {
    if (this.spriteSheet) {
        // Sprite Sheet frame size: pixels = total pixels / total frames
        const frameWidth = this.spriteData.pixels.width / this.spriteData.orientation.columns;
        const frameHeight = this.spriteData.pixels.height / this.spriteData.orientation.rows;

        // Sprite Sheet direction data source (e.g., front, left, right, back)
        const directionData = this.spriteData[this.direction];

        // Sprite Sheet x and y declarations to store coordinates of current frame
        let frameX, frameY;
        // Sprite Sheet x and y current frame: coordinate = (index) * (pixels)
        frameX = (directionData.start + this.frameIndex) * frameWidth;
        frameY = directionData.row * frameHeight;

        // Set up the canvas dimensions and styles
        this.canvas.width = frameWidth;
        this.canvas.height = frameHeight;
        this.canvas.style.width = `${this.width}px`;
        this.canvas.style.height = `${this.height}px`;
        this.canvas.style.position = 'absolute';
        this.canvas.style.left = `${this.position.x}px`;
        this.canvas.style.top = `${this.gameEnv.top+this.position.y}px`;

        // ... Transformations and animation omitted

        // Clear the canvas before drawing
        this.ctx.clearRect(0, 0, this.canvas.width, this.canvas.height);            

        // Update the frame index for animation at a slower rate
        this.frameCounter++;
        if (this.frameCounter % this.animationRate === 0) {
            this.frameIndex = (this.frameIndex + 1) % directionData.columns;
        }
    } else {
        // Draw default red square
        this.ctx.fillStyle = 'red';
        this.ctx.fillRect(0, 0, this.canvas.width, this.canvas.height);
    }
}
```

## Hacks
In initial development of this game it was intended to have some default behaviors.  Here are a couple of elements that should be revived.

1. A red square for player one and a green square for player two.  There are some definitions in the game to support alternatives.
2. Instead of static defintion of default direction as **down** it would be beneficial if the snowspeeder was facing right.
3. Idle animations was lost along the way of development and should be revived.
4. Instead of idle being no movement, it seems like snowspeeder or squid would need to be moving slightly according to their game environment.